... so we should expand the term in $\Gamma(\alpha/K)$, on the right hand side:

$$
\to_{K \to \infty}
\frac
  {K!}
  {\prod_{h=0}^{2^N-1}K_h!}
\left(\frac{\alpha}{K}\right)^K
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {(\alpha/K)\Gamma(N + 1)}
  \right)^{K_0}
$$

$$
=
\frac
  {K!}
  {\prod_{h=0}^{2^N-1}K_h!}
\left(\frac{\alpha}{K}\right)^K
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac{K}
  {\alpha}
\right)^{K_0}
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K_0}
$$

Gotcha number two, that got me: whilst a single term in $a + b/K$ will tend to $a$ as $K \to \infty$, the same term raised to a power of $K$, will not necessarily do so, and the infinite power of a term that is assymptotic to one must be handled together.

$$
\left(\frac{\alpha}{K} \right)^K \left( \frac{K}{\alpha} \right)^{K_0}
  = \left(\frac{\alpha}{K} \right)^{K^+}
$$

Giving:

$$
\to_{K \to \infty}
\alpha^{K^+}
\frac
  {K!}
  {K^{K^+}\prod_{h=0}^{2^N-1}K_h!}
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K_0}
 $$

Looking at the term:

$$\prod_{h=0}^{2^N-1}K_h!$$

$K_h \le K^+$ for $h>0$.  However, $K_0$ is not finite, so we should split that out separately:

$$
\to_{K \to \infty}
\alpha^{K^+}
\frac
  {K!}
  {K^{K^+}K_0!\prod_{h=1}^{2^N-1}K_h!}
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K_0}
$$

And we have:

$$\frac{K!}{K_0!K^{K^+}} \to 1$$

(from earlier).  So:

$$
\to_{K \to \infty}
\frac
  {\alpha^{K^+}}
  {\prod_{h=1}^{2^N-1}K_h!}
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(m_k)\Gamma(N-m_k+1}
  {\Gamma(N + 1)}
  \right)
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K_0}
$$


Now we need to deal with the right-hand term, the one raised to the power $K_0$.  Initially, I assumed this was assymptotic to $1$, from looking at the terms inside, which tend themselves to $1$.  However, after inspecting the answer in the tutorial, it looks like we need to handle the terms inside the brackets together with the infinite power of $K_0$.

The tutorial asserts that:

$$
\left(
  \frac{N!}
    {\prod_{j=1}^N(j + \alpha/K)}
\right)^K
\to
  \exp \left(-\alpha\sum_{j=1}^N 1/j \right)
$$

Let's start by trying to figure out this bit.  Is it using Stirling's approximation? Taylor series? Something else?

Taylor series includes things like:

$$
\exp(x) = \sum_{j=1}^{\infty} \frac{x^j}{j!}
$$

Stirling's approximation looks something like:

$$
n! \to \sqrt{2\pi n}\left( \frac{n}{e} \right)^n
$$

Let's try inverting the equivalence in the tutorial, to see if it looks more familiar:

$$
\left(
  \frac
    {\prod_{j=1}^N(j + \alpha/K)}
    {N!}
\right)^K
\to
  \exp \left(\alpha\sum_{j=1}^N 1/j \right)
$$

It sort of has characteristics of both Taylor series for $\exp(x)$ and Stirling's approximation, but not clear match with either :-P

- there's a $\pi$ term in the Stirling's approximation, which isnt present in the tutorial equivalence. And seems no obvious reason why the $\pi$ would disappear
- the Taylor series involves a sum over infinite terms, whereas the tutorial expression has a product, over a finite number of terms



Let's try taking logs, of the tutorial limit expression, to get rid of the power to K, and the $\exp(x)$  (note: this was just my hacking around, didnt actually work for me...)

$$
K\log \left( \prod_{j=1}^N ( j + \alpha/K ) \right)
- K\log(N!)
\to
\alpha\sum_{j=1}^N \frac{1}{j}
$$

So:
$$
K\sum_{j=1}^N \log(j + \alpha/K)
- K\log(N!)
\to
\alpha\sum_{j=1}^N \frac{1}{j}
$$

Writing the log of the factorial also as a sum of logs:

$$
K\sum_{j=1}^N \log(j + \alpha/K)
- K\sum_{j=1}^N(\log(j))
\to
\alpha\sum_{j=1}^N \frac{1}{j}
$$

Looking at just the left hand side for now:

$$
K\sum_{j=1}^N \log(j + \alpha/K)
- K\sum_{j=1}^N(\log(j))
$$
$$
=
K\sum_{j=1}^N (\log(j+\alpha/K) - \log(j))
$$

$$
=
K\sum_{j=1}^N \log \left( 1 + \frac{\alpha}{jK} \right)
$$


Ok, at this point, I gave up, and looked at the appendix.  The identity that we want is:

$$
\exp(x) = \lim_{n \to \infty} \left(1 + \frac{x}{n} \right)^n
$$

which in our case becomes, by inverting both sides:

$$
\lim_{K \to \infty}
\left(
\frac{1}
  {1 + x\,/\,K}
\right)^K
= \frac{1}{\exp(x)}
$$

So, looking at:

$$
\left(
  \frac{N!}
    {\prod_{j=1}^N(j + \alpha/K)}
\right)^K
$$

This is:

$$
\left(
  \frac{\prod_{j=1}^Nj}
    {\prod_{j=1}^N(j + \alpha/K)}
\right)^K
$$
$$
=
\left(
\prod_{j=1}^N
  \frac{j}
    {(j + \alpha/K)}
\right)^K
$$
$$
=
\left(
\prod_{j=1}^N
  \frac{1}
    {1 + \frac{\alpha}{jK}}
\right)^K
$$


$$
(xyz)^a
$$
$$
=x^ay^az^a
$$
So, $$
\left( \prod_{i=1}^N x_i \right)^a
$$
$$
= \prod_{i=1}^N x_i^a
$$

So,
$$
\left(
\prod_{j=1}^N
  \frac{1}
    {1 + \frac{\alpha}{jK}}
\right)^K
$$
$$
=
\prod_{j=1}^N
\left(
  \frac{1}
    {1 + \frac{\alpha}{jK}}
\right)^K
$$

Using identity from above:
$$
\lim_{K \to \infty} 
\prod_{j=1}^N
\left(
  \frac{1}
    {1 + \frac{\alpha}{jK}}
\right)^K
$$

$$
=\prod_{j=1}^N
\left(
\frac{1}
  {\exp(\alpha/j)}
\right)
$$

$$
=\prod_{j=1}^N \exp \left(-\frac{\alpha}{j}\right)
$$

$$
= \exp \left(-\sum_{j=1}^N \frac{\alpha}{j}\right)
$$

$$
= \exp \left(-\alpha \sum_{j=1}^N \frac{1}{j}\right)
$$

... as required.